In [2]:
import deepSI_lite as dsi
import torch

## Port Hamiltonian Neural Networks in `deepSI_lite`

Model structure given by `pHNN_SUBNET`

$$ \frac{dx}{dt} = \frac{1}{\tau} \left (\left ( J(x) - R(x) \right ) \frac{dH}{dx} + G(x) (u - u_\text{mean})/u_\text{std} \right)$$

$$ (y - y_\text{mean})/y_\text{std} = G(x)^T \frac{dH}{dx}  $$

where 
$$G (\text{Gnet}) : n_\text{x} \rightarrow n_\text{x} \times n_\text{u}$$
$$J (\text{Jnet}) : n_\text{x} \rightarrow n_\text{x} \times n_\text{x}\ \text{(skew symetric)}$$
$$R (\text{Rnet}) : n_\text{x} \rightarrow n_\text{x} \times n_\text{x}\ \text{(semi positive def)}$$
$$H (\text{Hnet}) : n_\text{x} \rightarrow\ \text{scalar}$$
$$u_\text{mean},\ u_\text{std},\ y_\text{mean},\ y_\text{std}\ \text{given by the `norm.umean`, `norm.ustd`, ect.}$$
also `model.integrator(f, x, u, dt)` is a function that integrates the state given a certain state derivative $f$ and input $u$ for $dt$ time. 

These function are constructed by default by using `MLP_res_net` as a base and than converting the output such that it adhers to the constraints. 

In [4]:
norm = dsi.normalization.Norm(0,1,0,1)
na = nb = 3
nx = 4
nu = ny = 'scalar'
model = dsi.models.pHNN_SUBNET(nu, ny, norm, nx, na, nb)

# net(torch.randn(3,4)).shape
r = torch.randn
b = 2
T = 10
upast, ypast, ufuture, yfuture = r(b, nb), r(b, na), r(b, T), r(b, T)
sampling_time = r(b)
model(upast, ypast, ufuture, sampling_time=sampling_time)

tensor([[-0.4271, -0.4314, -0.4297, -0.4335, -0.4365, -0.4358, -0.4374, -0.4383,
         -0.4407, -0.4442],
        [ 0.1740,  0.1735,  0.1715,  0.1729,  0.1718,  0.1719,  0.1698,  0.1701,
          0.1699,  0.1709]], grad_fn=<AddBackward0>)

## Customized function for each element: 

Example (this will get expanded later)

In [5]:
import deepSI_lite as dsi
import torch
from torch import nn
from deepSI_lite.networks import Quadratic_net

class Sym_pos_semidef_converter(nn.Module):
    def __init__(self, net, norm='auto'):
        super().__init__()
        self.norm = norm
        self.net = net

    def forward(self, x):
        z = self.net(x)
        nx = int(round(z.shape[1]**0.5))
        assert nx*nx==z.shape[1], 'the output of net needs to have a sqaure number of elements to be reshaped to a square matrix'
        A = z.view(z.shape[0], nx, nx)
        if self.norm=='auto':
            A = A/(((nx+2)*nx**2)**0.25) #this might not be entirely correct
        else:
            A = A/self.norm
        R = torch.einsum('bik,bjk->bij', A, A)
        return R

nx = 4
Jnet_bias = dsi.networks.Bias_net(nx*nx)
Jnet_constant = dsi.networks.Contant_net(torch.randn(nx*nx))
Jnet_mlp = dsi.networks.MLP_res_net(input_size=nx, output_size=nx*nx) #simple 
Jnet = dsi.networks.Sum_net([Jnet_bias, Jnet_constant, Jnet_mlp]) #add these three networks together
Jnet = dsi.networks.Skew_sym_converter(Jnet) #x -> nx x nx 
Rnet = dsi.networks.Bias_net(nx*nx)
Rnet = dsi.networks.Sym_pos_semidef_converter(Rnet)

Hnet_depend = dsi.networks.ELU_lower_bound(dsi.networks.MLP_res_net(nx, 'scalar'), lower_bound=-100)
Hnet_qaudratic = Quadratic_net(nx)
Hnet = dsi.networks.Ham_converter(dsi.networks.Sum_net([Hnet_depend,Hnet_qaudratic]))

nu = 'scalar'
ny = 'scalar'

norm = dsi.normalization.Norm(0,1,0,1)
na = nb = 3
model = dsi.models.pHNN_SUBNET(nu, ny, norm, nx, na, nb, Jnet=Jnet, Rnet=Rnet, Hnet=Hnet)

# net(torch.randn(3,4)).shape
r = torch.randn
b = 2
T = 10
upast, ypast, ufuture, yfuture = r(b, nb), r(b, na), r(b, T), r(b, T)
sampling_time = r(b)
model(upast, ypast, ufuture, sampling_time=sampling_time).shape

torch.Size([2, 10])